In [18]:
import torch

one_by_three = torch.arange(2, 5, dtype = torch.float32).reshape(1, 3)
three_by_three = torch.arange(9, dtype = torch.float32).reshape(3, 3)

print(one_by_three)
print(three_by_three)

tensor([[2., 3., 4.]])
tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])


In [19]:
one_by_three.size()

torch.Size([1, 3])

In [20]:
one_by_three * three_by_three

tensor([[ 0.,  3.,  8.],
        [ 6., 12., 20.],
        [12., 21., 32.]])

In [ ]:
tup1 = (2,)

In [21]:
import numpy as np

numpy_arr = np.array([1., 2., 3.])
torch_tensor = torch.tensor([1., 1., 1.])

numpy_arr - torch_tensor

TypeError: unsupported operand type(s) for -: 'numpy.ndarray' and 'Tensor'

In [22]:
torch.pow(one_by_three, 2)

tensor([[ 4.,  9., 16.]])

In [26]:
torch.sum(three_by_three, [0, 1])

tensor(36.)

In [32]:
import torch.nn as nn
from typing import *

class DiagGaussian(nn.Module):
    # Multivariate Gaussian distribution with diagonal covariance matrix

    def __init__(self,
                 shape:Union[int, list],
                 trainable:bool=True) -> None:
        
        super().__init__()
        if isinstance(shape, int):
            shape = (shape,)
        if isinstance(shape, list):
            shape = tuple(shape)
        self.shape = shape
        self.n_dim = len(shape)
        self.d = np.prod(shape)
        if trainable:
            self.loc = nn.Parameter(torch.zeros(1, *self.shape))
            self.log_scale = nn.Parameter(torch.zeros(1, *self.shape))
        else:
            self.register_buffer("loc", torch.zeros(1, *self.shape))
            self.register_buffer("log_scale", torch.zeros(1, *self.shape))
        self.temperature = None  # Temperature parameter for annealed sampling

    def forward(self,
                num_samples:int=1,
                context=None) -> tuple[torch.Tensor, torch.Tensor]:
        
        eps = torch.randn(
            (num_samples,) + self.shape, dtype=self.loc.dtype, device=self.loc.device
        )
        if self.temperature is None:
            log_scale = self.log_scale
        else:
            log_scale = self.log_scale + np.log(self.temperature)
        z = self.loc + torch.exp(log_scale) * eps
        summed = torch.sum(log_scale + 0.5 * torch.pow(eps, 2), list(range(1, self.n_dim + 1)))
        log_p = -0.5 * self.d * np.log(2 * np.pi) - summed
        return eps, log_scale, summed, z, log_p

In [45]:
dg = DiagGaussian(shape = 3)
dg.loc

Parameter containing:
tensor([[0., 0., 0.]], requires_grad=True)

In [46]:
eps, log_scale, summed, z, log_p = dg(num_samples = 7)
eps

tensor([[ 1.8793e+00, -9.8875e-01,  1.5219e+00],
        [ 3.1710e-01,  1.7754e+00,  1.1747e+00],
        [-1.6328e+00, -7.1647e-04,  1.6360e-01],
        [ 3.9588e-02, -1.4581e-02,  7.9325e-01],
        [-1.9799e+00,  3.4490e-03, -5.9534e-01],
        [ 1.6258e-01, -1.2493e+00,  9.2686e-01],
        [-2.5759e-01,  1.1668e+00, -1.5670e+00]])

In [47]:
0.5 * torch.pow(eps, 2)

tensor([[1.7658e+00, 4.8881e-01, 1.1582e+00],
        [5.0276e-02, 1.5760e+00, 6.8994e-01],
        [1.3330e+00, 2.5667e-07, 1.3383e-02],
        [7.8361e-04, 1.0630e-04, 3.1462e-01],
        [1.9600e+00, 5.9480e-06, 1.7721e-01],
        [1.3216e-02, 7.8041e-01, 4.2953e-01],
        [3.3175e-02, 6.8068e-01, 1.2278e+00]])

In [48]:
summed

tensor([3.4128, 2.3163, 1.3464, 0.3155, 2.1372, 1.2232, 1.9416],
       grad_fn=<SumBackward1>)